# Notebook for testing data for pilot 3

Pilot 3 added more checks for engagement early in the experiment.
Iteration names:
'pre-pilot3':

## 1. Read in packages and set up server connection
This first section will read in necessary packages for anaysis and establish file paths and connections to the mongo db server

In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import drawing_utils as drawing
import importlib
import scoring

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional-abstractions']
coll = db['two-towers']

# which iteration name should we use?
iterationName = 'pilot3'

## look up number of trials (including paired-practice)
numTrials = 13

In [ ]:
# look into failType

## 2. Construct tidy dataframe with game data

### establish connection to mongo
First thing you need to do is to establish an ssh tunnel (aka remote port forwarding) to the server, so that requests to the mongodb can be made "as if" the mongodb server is running on your local computer. Run this from the command line before you begin data analysis if you plan to fetch data from mongo:

`ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org`

#### Pass in list of iterations, returns list of total games and completed games for each

In [4]:
## get list of all gameIDs in database
#iterationList = ['livetest0','pilot0','pilot2']

iterationList = ['pilot3']

for iteration in iterationList:
    #get total games
    total_games = coll.find({'iterationName':iteration}).distinct('gameid')
    print('There are {} total games in iteration: {}.'.format(len(total_games), iteration))

    ## get list of complete gameIDs
    gameIDs = coll.find({'iterationName':iteration}).distinct('gameid')
    complete_games = [g for g in gameIDs if len(coll.find({'gameid':g}).distinct('trialNum')) == numTrials]
    print('There are {} complete games in iteration: {}.'.format(len(complete_games), iteration))

There are 2 total games in iteration: pilot3.
There are 2 complete games in iteration: pilot3.


In [5]:
broken_games = []
complete_games = [g for g in complete_games if g not in broken_games]

def construct_tidy_dataframe(eventType = 'chatMessage', 
                             complete_games = [],
                             iterationName = 'pilot3',
                             remove_workerID = True):
    '''
    input: list of complete games and name of event Type
    '''
    event2name = {'chatMessage':'chat', 'block':'block', 'endTrial':'trial', 'exitSurvey':'exit'}
    L = pd.DataFrame()
    for g, this_gameID in enumerate(complete_games):
        print('Analyzing game {} | {} of {}'.format(this_gameID, g+1, len(complete_games)))
        clear_output(wait=True) 

        ### extract records 
        #loop over iteration names??
        X = coll.find({ '$and': [{'iterationName':iterationName}, 
#                                  {"$or":[{'iterationName':'Exp2Pilot3'},
#                                  {'iterationName':'Exp2Pilot3_batch2'}]}
                                 {'gameid': this_gameID}, {'eventType': eventType}]}).sort('time') 
        
        li = list(X)        
        _L = pd.DataFrame(li)  

        ## concat with previous game's dataframe
        if L.shape[0]==0:
            L = _L
        else: 
            L = pd.concat([L,_L], axis=0)     

    ## postprocessing
    if remove_workerID and 'workerId' in L.columns:
        L = L.drop('workerId',axis=1)
        
    if eventType in ['block','endTrial','chatMessage']:
        L['practice'] = L.trialNum == 'practice'
        L.trialNum = pd.to_numeric(L['trialNum'], errors='coerce')

    ## save out group dataframe to csv dir
    out_path = os.path.join(csv_dir,'compabs_{}_{}.csv'.format(event2name[eventType],iterationName))
    print('Saving dataframe out to CSV dir at path: {}'.format(out_path))    
    L.to_csv(out_path)             

    return L


In [6]:
## construct dataframe for each datatype
dataTypes = coll.distinct('eventType')
for thisDataType in dataTypes:
    X = construct_tidy_dataframe(eventType=thisDataType, complete_games=complete_games, iterationName=iterationName)


Saving dataframe out to CSV dir at path: /Users/will/compositional-abstractions/results/csv/compabs_exit_pilot3.csv


## 3. Create a full DF from DB
We can read in a full df from the db and do some basic checks to make sure the data look right

#### check outcome for specific workerID
Must pass in workerID and corresponding iteration

In [169]:
f = pd.DataFrame(coll.find({'iterationName': 'pilot3'}))
f['gameid'].unique()

array(['5809-2a5e096b-a425-4bc7-b495-cf74cb60ded6',
       '6598-80d65299-0654-43b4-9f27-08572427cb2d'], dtype=object)

In [168]:
a = pd.DataFrame(coll.find({'failType': 'instructionsQuiz'}))
a

,_id,workerId,failType,iterationName
0,5f47c827df57f9222c29cc11,A21WWDLXGJQHWZ,instructionsQuiz,NaN
1,5f47c83fdf57f9222c29cc12,A2UUZ3WDQBQ21A,instructionsQuiz,NaN
2,5f47d778df57f9222c29cc22,A3ENDV5PRQI2U2,instructionsQuiz,NaN
3,5f47d871df57f9222c29cc29,AQ8B4BC1EUWN,instructionsQuiz,NaN
4,5f47def1df57f9222c29ccbb,A2K4X1QPAZ0DZX,instructionsQuiz,NaN
5,5f4d42543c9d171947610dad,undefined,instructionsQuiz,pilot3_debug


In [165]:
b = pd.DataFrame(coll.find({'failType': 'soloPractice'}))
b

,_id,workerId,failType,iterationName
0,5f4d42293c9d171947610dac,undefined,soloPractice,pilot3_debug


In [160]:
c = pd.DataFrame(coll.find())

c[c.pairedPracticeFails==3]

,_id,iterationName,gameid,time,workerId,assignmentId,leftTarget,rightTarget,trialNum,turnNum,...,strategy,role,totalLength,cumulativeScore,cumulativeBonus,trialScore,workerID,individualPracticeAttempts,failType,pairedPracticeFails
6754,5f4d40de3c9d171947610dab,pilot3_debug,1068-ffe5a500-3e57-41ef-ac07-79b48edfcab3,1.598898e+12,,,vertical,horizontal,NaN,0.0,...,aervwav,listener,30657.0,NaN,NaN,NaN,NaN,None,NaN,3.0


## 4. Trial Level Data
We're going to construct a spearate df for each event type (chat, block, endTrial, Survey) which will make analysis a little easier.
We also want to be able to concatenate multiple iterations for grouped analysis of previous versions.

### Create DF for each event


In [8]:
event_types = ['block', 'chat','exit','trial']
iteration_list = ['pilot3']
df_dict = {}
for event in event_types:
    event_dict = {}
    df_name = 'df_'+ event
    for iteration in iteration_list:
        df_temp = 'df_'+event+"_"+iteration # the name for the dataframe
        file_name = '../results/csv/compabs_{}_{}.csv'.format(event,iteration)
        event_dict[df_temp] = pd.read_csv(file_name)
    df_dict[df_name] = pd.concat(event_dict.values(), ignore_index = True)

##### Light data cleaning

In [98]:
#create columns for char and word counts
df_chat['word_count'] = df_chat['content'].str.split(' ').str.len()
df_chat['char_count'] = df_chat['content'].str.len()
df_chat["timeElapsedInTurn"] = pd.to_numeric(df_chat['timeElapsedInTurn'])

In [ ]:
df_block['w'] = df_block['width']
df_block['h'] = df_block['height']
df_block["timeElapsedInTurn"] = pd.to_numeric(df_block['timeElapsedInTurn'])

#### Write to data frames

In [9]:
df_dict['df_block'].to_csv('df_block.csv', header=True, index=False) 
df_dict['df_chat'].to_csv('df_chat.csv', header=True, index=False) 
df_dict['df_exit'].to_csv('df_exit.csv', header=True, index=False) 
df_dict['df_trial'].to_csv('df_trial.csv', header=True, index=False) 

In [10]:
#read in dataframes from each eventType
df_block = pd.read_csv('df_block.csv')
df_chat = pd.read_csv('df_chat.csv')
df_exit = pd.read_csv('df_exit.csv')
df_trial = pd.read_csv('df_trial.csv')